In [23]:
import tensorflow as tf
from tensorflow import keras
from keras.utils import image_dataset_from_directory
from keras.preprocessing import image
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input, decode_predictions
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten

In [24]:
# Loading training and validation data 
train_data = image_dataset_from_directory(
    directory="../data/raw/train/images",
    label_mode = 'categorical',
    color_mode='rgb',
    image_size=(224, 224),
    batch_size=32
)

test_data = image_dataset_from_directory(
    directory="../data/raw/validation/images",
    label_mode = 'categorical',
    color_mode='rgb',
    image_size=(224, 224),
    batch_size=32
)

Found 1440 files belonging to 6 classes.
Found 360 files belonging to 6 classes.


In [25]:
# NORMALIZING THE IMAGES

def preprocess(image, label):
    image = tf.image.resize(image, (224, 224))
    image = preprocess_input(image)
    return image, label

train_input = train_data.map(preprocess)
test_input = test_data.map(preprocess)

In [26]:
AUTOTUNE = tf.data.AUTOTUNE

train_input = train_input.cache().prefetch(buffer_size=AUTOTUNE)
test_input = test_input.cache().prefetch(buffer_size=AUTOTUNE)

In [27]:
# Data Augmentation
augment_step = Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomFlip("vertical"),
    tf.keras.layers.RandomRotation(0.05),
    tf.keras.layers.RandomZoom(0.1)
])

In [28]:
base_model = MobileNetV2(weights='imagenet', include_top='false', input_shape=(224, 224,3))

In [29]:
base_model.trainable = False

In [31]:
model = Sequential()

model.add(keras.layers.Input(shape=(224, 224,3)))

model.add(augment_step)

model.add(base_model)

model.add(Dense(6, activation='softmax'))

In [32]:
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_4 (Sequential)       │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 1000)           │     3,538,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 6)              │         6,006 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,544,990 (13.52 MB)

 Trainable params: 6,006 (23.46 KB)

 Non-trainable params: 3,538,984 (13.50 MB)

In [33]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])

In [34]:
history = model.fit(train_input, epochs=10, validation_data=test_input)

Epoch 1/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 58s 999ms/step - accuracy: 0.3396 - loss: 1.7837 - val_accuracy: 0.4639 - val_loss: 1.7758
Epoch 2/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 39s 863ms/step - accuracy: 0.6479 - loss: 1.7687 - val_accuracy: 0.6139 - val_loss: 1.7608
Epoch 3/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 39s 864ms/step - accuracy: 0.7340 - loss: 1.7537 - val_accuracy: 0.6556 - val_loss: 1.7466
Epoch 4/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 39s 863ms/step - accuracy: 0.7771 - loss: 1.7383 - val_accuracy: 0.6667 - val_loss: 1.7322
Epoch 5/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 39s 861ms/step - accuracy: 0.7806 - loss: 1.7250 - val_accuracy: 0.6694 - val_loss: 1.7185
Epoch 6/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 39s 869ms/step - accuracy: 0.7688 - loss: 1.7118 - val_accuracy: 0.6833 - val_loss: 1.7041
Epoch 7/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 39s 867ms/step - accuracy: 0.7958 - loss: 1.6975 - val_accuracy: 0.6833 - val_loss: 1.6899
Epoch 8/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 39s 869ms/step - accuracy: 0.7917 - loss: 1.6841 - val_accu

In [11]:
print(history.history.keys())

dict_keys(['accuracy', 'loss'])
